In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

In [2]:
conn = pymongo.MongoClient()
wine_db = conn.wine
wine_collection = wine_db.wine_collection

In [3]:
wine_url = 'http://www.keumyang.com/mall/KY_searchNew.ky?wine_select=&wine_country=&wine_winery=&wine_rape=&windeTxt=&lang=ko&ps_page=&ps_page={}'
wine_info_list = list()
for i in range(1,44):
    url = wine_url.format(i)
    res = requests.post(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    wines = soup.select('div.wine_roll_normal div')
    for wine in wines:
        wine_info_dict = dict()
        wine.select_one('a').attrs['href']
        wine_ko_name = wine.select_one('span.title').text
        wine_link = 'http://www.keumyang.com/mall' + wine.select_one('a').attrs['href'][1:] 
        response_wine = requests.get(wine_link)
        soup_wine = BeautifulSoup(response_wine.content, 'html.parser')
        detail_top = soup_wine.select_one('div.detail_top')
        wine_image = detail_top.select_one('div.detail_img').select_one('img').attrs['src'][1:]
        wine_image_url = 'http://www.keumyang.com/mall' + wine_image
        wine_info_dict['wine_image'] = wine_image_url
        wine_en_name = detail_top.select_one('strong').text
        wine_info_dict['wine_ko_name'] = wine_ko_name
        wine_info_dict['wine_en_name'] = wine_en_name
        information = detail_top.select('dd li')
        for info in information:
            wine_info_field = info.select('span.title')[0].text.replace(" ","")
            wine_info_value = info.select('span.txt')[0].text
            if wine_info_field == 'BOTTLE\xa0\xa0SIZE':
                wine_info_field = 'SIZE'
            if wine_info_field == 'REGION':
                wine_info_value = info.select('span.txt')[0].text.replace("\t\t\t\t","")
                wine_info_value = wine_info_value.split('>')
                strip_list = list()
                for i in wine_info_value:
                    strip_list.append(i.replace(" ",""))
                wine_info_value = strip_list
            if wine_info_field == 'FOOD':
                wine_info_value = info.select('span.txt')[0].text.split('/ ')
            wine_info_dict[wine_info_field] = wine_info_value
        tasting_note = soup_wine.select_one('div.detail_p').text
        try:
            tasting_sweetness = soup_wine.select('div.detail_tasting span.circle')[0].select('span')[0].text
            tasting_body = soup_wine.select('div.detail_tasting span.circle')[1].select('span')[0].text
            wine_info_dict['tasting_note'] = tasting_note
            wine_info_dict['tasting_sweetness'] = int(tasting_sweetness)
        except IndexError:
            None
        wine_info_dict['tasting_body'] = int(tasting_body)
        wine_info_list.append(wine_info_dict)
wine_info_list

[{'wine_image': 'http://www.keumyang.com/mall/shop_image_new/201909/2_1567987550_PressHouseVidalIcewine_1567987550.png',
  'wine_ko_name': '프레스 하우스 비달 아이스와인',
  'wine_en_name': 'Press House Vidal Icewine',
  'BRAND': 'Canada',
  'REGION': ['캐나다', '온타리오주'],
  'VARIETAL': '비달 100%',
  'CATEGORIES': '화이트와인',
  'ALCOHOL': '11%',
  'TEMP': '7-10℃',
  'SIZE': '375ml',
  'FOOD': ['해산물', '흰 살 생선', '채소', '디저트', '과일'],
  'tasting_note': '2014년 1월 영하 9도에서 얼어있는 상태의 포도를 손 수확하여 양조하고 극한 기후 조건에서 만들어지는 아이스와인의 진수를 맛 볼 수 있다.',
  'tasting_sweetness': 5,
  'tasting_body': 1},
 {'wine_image': 'http://www.keumyang.com/mall/shop_image_new/201909/2_1567987421_CANADA150VIDALICEWINE_1567987421.png',
  'wine_ko_name': '캐나다 150 비달 아이스 와인',
  'wine_en_name': 'CANADA 150 VIDAL ICE WINE',
  'BRAND': 'Canada',
  'REGION': ['캐나다', '온타리오주'],
  'VARIETAL': '비달 100%',
  'CATEGORIES': '화이트와인',
  'ALCOHOL': '11%',
  'TEMP': '7-10℃',
  'SIZE': '375ml',
  'FOOD': ['해산물', '흰 살 생선', '채소', '디저트', '과일'],
  'tasting_note': '2014년 

In [4]:
wine_collection.insert_many(wine_info_list)